In [1]:
import keras
from keras.datasets import mnist
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
(x_train_origin, y_train_origin),(x_test_origin, y_test_origin) = mnist.load_data()
x_train, x_test = x_train_origin / 255.0, x_test_origin / 255.0

In [3]:
# if digit == 1, then label = 1, else label = 0
func = lambda x: 1 if x == 1 else 0
#y_train = list(map(func, y_train_origin))
#y_test = list(map(func, y_test_origin))

In [4]:
x_train = x_train_origin.reshape((60000, 28 * 28))
x_train = x_train_origin.astype('float32') / 255

x_test = x_test_origin.reshape((10000, 28 * 28))
x_test = x_test_origin.astype('float32') / 255

In [5]:
y_train = np.array([func(y) for y in y_train_origin])
y_test = np.array([func(y) for y in y_test_origin])

In [6]:
#y_train = y_train.reshape(1,x_train.shape[0])
#y_test = y_test.reshape(1,x_test.shape[0])
#y_train = y_train.T
#y_test = y_test.T

In [7]:
x_train = x_train.reshape(x_train.shape[0], -1).T
x_test = x_test.reshape(x_test.shape[0], -1).T

y_train = y_train.reshape(1,y_train.shape[0])
y_test = y_test.reshape(1,y_test.shape[0])

In [8]:
def sigmoid(z):
    
    s = 1.0/(1.0 + np.exp(-z))
    
    return s

In [9]:
def initialize_params_with_zeros(dim):
    
    w = np.zeros((dim,1))
    b = 0
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [10]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]

    A = sigmoid(np.dot(w.T, X) + b)                                       
    loss = (1/m) * np.sum(np.square(Y - A))
    
    dL = 2*(A - Y)
    dA = A * (1 - A) * dL
    
    #dw = (1/m)*np.dot(X,dA.T)
    #db = 1/m*np.sum(dA)
    
    dw = (1/m) * np.dot(X, (A - Y).T)
    db = (1/m) * np.sum(A - Y)

    #cost = np.squeeze(cost)
    #assert(cost.shape == ())
    grads = {"dw": dw,
             "db": db}
    return grads, loss

In [11]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]

    A = sigmoid(np.dot(w.T, X) + b)                                       
    cost = -1./m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1-A)) 

    dw = (1/m)*np.dot(X,(A-Y).T)
    db = 1/m*np.sum(A-Y)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)

    cost = np.squeeze(cost)
    assert(cost.shape == ())
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [12]:
def optimize(w, b, X, Y, num_iters, learning_rate, print_cost=False):
   
    costs = []
    
    for i in range(num_iters):
        
        grads, cost = propagate(w, b, X, Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate *  dw    
        b = b - learning_rate *  db
        
        costs.append(cost)
        
        print ("Cost after iteration %i: %f" % (i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [13]:
# GRADED FUNCTION: predict

def predict(w, b, X):
   
    m = X.shape[1]
    Y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    

    A = sigmoid(np.dot(w.T, X) + b)

    for i in range(A.shape[1]):
        if A[0][i] <= 0.5:
            Y_pred[0, i] = 0
        else:
            Y_pred[0, i] = 1
                
    assert(Y_pred.shape == (1, m))
    
    return Y_pred

In [14]:
# GRADED FUNCTION: model
def model(x_train, y_train, x_test, y_test, num_iters=2000, learning_rate=0.5, print_cost=False):
    

    w, b = initialize_params_with_zeros(x_train.shape[0])                          

    parameters, grads, costs = optimize(w, b, x_train, y_train, num_iters, learning_rate, print_cost=True)     

    w = parameters["w"]
    b = parameters["b"]
    

    Y_pred_test = predict(w, b, x_test)
    Y_pred_train = predict(w, b, x_train)

    print("")

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_pred_train - y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_pred_test - y_test)) * 100))
    repr('train accuracy')


    
    d = {"costs": costs,
         "Y_pred_test": Y_pred_test, 
         "Y_pred_train" : Y_pred_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iters": num_iters}
    
    return d

In [54]:
d = model(x_train, y_train, x_test, y_test, num_iters=10, learning_rate=0.5, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 1: 0.511074
Cost after iteration 2: 0.336341
Cost after iteration 3: 0.194794
Cost after iteration 4: 0.122501
Cost after iteration 5: 0.104322
Cost after iteration 6: 0.098650
Cost after iteration 7: 0.094533
Cost after iteration 8: 0.091016
Cost after iteration 9: 0.087949

train accuracy: 98.095 %
test accuracy: 98.21 %


In [55]:
d = model(x_train, y_train, x_test, y_test, num_iters=5, learning_rate=0.5, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 1: 0.511074
Cost after iteration 2: 0.336341
Cost after iteration 3: 0.194794
Cost after iteration 4: 0.122501

train accuracy: 97.25833333333334 %
test accuracy: 97.4 %


In [15]:
d = model(x_train, y_train, x_test, y_test, num_iters=10, learning_rate=0.5, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 1: 0.511074
Cost after iteration 2: 0.336341
Cost after iteration 3: 0.194794
Cost after iteration 4: 0.122501
Cost after iteration 5: 0.104322
Cost after iteration 6: 0.098650
Cost after iteration 7: 0.094533
Cost after iteration 8: 0.091016
Cost after iteration 9: 0.087949

train accuracy: 98.095 %
test accuracy: 98.21 %
